In [104]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm

In [ ]:
driver = webdriver.Firefox()
ref = 'https://lapsha.media/novoe/'
driver.get(ref)
time.sleep(30)
button = driver.find_element(By.CLASS_NAME, 'closebanner')
button.click()
time.sleep(0.5)
button = driver.find_element(By.CLASS_NAME, 'cn-buttons-container')
button.click()
time.sleep(0.5)
for _ in range(1000):
    time.sleep(0.5)
    button = driver.find_element(By.CLASS_NAME, 'lenta-loader')
    button.click()
# пока что падает с ошибкой после нескольких минут прокрутки страницы, но работу свою делает

In [41]:
tree = BeautifulSoup(driver.page_source, 'html.parser')

In [43]:
lenta = tree.find('div',{'class':'lenta'})

In [60]:
links_to_articles = []
for post in lenta:
    if post.find('a', {'class': 'postblock-title__tag'}).text == 'Фейки':
        link = post.find('meta').get('content')
        links_to_articles.append(link)

In [66]:
links_to_articles[:10]

['https://lapsha.media/feiky/v-seti-pojavilis-kadry-obrushenija-paninskogo-mosta-v-smolenskoj-oblasti/',
 'https://lapsha.media/feiky/transljaciju-kanala-rossija-1-v-rjazani-vzlomali/',
 'https://lapsha.media/feiky/shkolnikov-budut-otpravljat-v-zonu-svo-vmesto-letnih-kanikul/',
 'https://lapsha.media/feiky/ramzan-kadyrov-trebuet-uluchshit-uslovija-soderzhanija-terroristov-iz-krokusa/',
 'https://lapsha.media/feiky/rossijane-mogut-ne-platit-za-uslugi-zhkh/',
 'https://lapsha.media/feiky/detej-tajno-vakcinirujut-s-pomoshhju-kapel-grippferon/',
 'https://lapsha.media/feiky/v-stavropole-testirujut-sistemu-opoveshhenija-iz-za-massovoj-ataki-dronov/',
 'https://lapsha.media/feiky/v-mchs-predupredili-o-sbrose-vody-na-juzhnouralsk-nochju-7-aprelja/',
 'https://lapsha.media/feiky/rossija-planiruet-napadenie-na-kazahstan/',
 'https://lapsha.media/feiky/it-specialisty-massovo-pokidajut-rossiju/']

In [62]:
len(links_to_articles)

3565

In [101]:
def get_article_text(url):
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    tree = BeautifulSoup(response.content, 'html.parser')
    title = tree.find('h1').text
    body = tree.find('div', {'class': 'bialty-container'})
    text = [title]
    for p in body:
        if 'Как на самом деле' in p.text:
            break
        text.append(p.text)
    return ' '.join(text)

In [102]:
text = get_article_text('https://lapsha.media/feiky/v-seti-pojavilis-kadry-obrushenija-paninskogo-mosta-v-smolenskoj-oblasti/')

In [103]:
text

'В сети появились кадры обрушения Панинского моста в Смоленской области В сети распространяют фейковое видео обрушения моста в Смоленской области.  Публикации с роликом преимущественно распространялись в украинских каналах. https://lapsha.media/wp-content/uploads/2024/04/2024-04-08-19.12.14.mp4'

In [106]:
df = pd.DataFrame(columns = ['text'])
for link in tqdm(links_to_articles):
    df.loc[len(df)] = get_article_text(link)

100%|██████████████████████████████████████████████████████████████████████████████| 3565/3565 [55:52<00:00,  1.06it/s]


In [108]:
df.to_csv('lapsha.csv', index = False)

In [119]:
df

,text
0,В сети появились кадры обрушения Панинского мо...
1,Трансляцию канала «Россия 1» в Рязани взломали...
2,Школьников будут отправлять в зону СВО вместо ...
3,Рамзан Кадыров требует улучшить условия содерж...
4,Россияне могут не платить за услуги ЖКХ В сети...
...,...
3560,Сибирский лес отдали китайцам в аренду С чего ...
3561,В таблетках парацетамола содержится смертельны...
3562,В воду будут добавлять антибиотики С чего все ...
3563,В Томской области захоронят тысячу тонн радиоа...
